In [1]:
from transformers import pipeline
import soundfile as sf
from transformers import AutoProcessor, MusicgenMelodyForConditionalGeneration, MusicgenForConditionalGeneration
from diffusers import StableAudioPipeline
import numpy as np
import sys, torch
from tqdm import tqdm as notebook_tqdm

MAX_TOKENS: int = 1503 # 30 seconds of tokens

/home/evanmm3/ms-research/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
_ = stableaudio_open(text_prompt='smooth jazz, soprano saxophone', time=15, outname='sax')

In [2]:
pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# define the prompts
prompt = "The sound of a hammer hitting a metal surface."
negative_prompt = "Low quality."

# set the seed for generator
generator = torch.Generator("cuda").manual_seed(0)

num_waveforms: int = 3

# run the generation
audio = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=200,
    audio_end_in_s=10.0,
    num_waveforms_per_prompt=3,
    generator=generator,
).audios

for i in range(num_waveforms):
    output = audio[i].T.float().cpu().numpy()
    sf.write(f"sounds/hammer{i}.wav", output, pipe.vae.sampling_rate)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]/home/evanmm3/ms-research/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
  0%|          | 0/200 [00:00<?, ?it/s]/home/evanmm3/ms-research/venv/lib/python3.10/site-packages/torchsde/_brownian/brownian_interval.py:608: UserWarning: Should have tb<=t1 but got tb=500.00006103515625 and t1=500.0.
  warnings.warn(f"Should have {tb_name}<=t1 but got {tb_name}={tb} and t1={self._end}.")
 99%|█████████▉| 198/200 [00:48<00:00,  7.63it/s]/home/evanmm3/ms-research/venv/lib/python3.10/site-packages/torchsde/_brownian/brownian_interval.py:599: UserWarning: Should have ta>=t0 but got ta=0.29999998211860657 and t0=0.3.
  warnings.warn(f"Should have ta>=t0 but got ta={ta} and t0={self._start}.")
100%|█████████▉| 199/200 [00:48<00:00,  7.61it/s]/home/evanmm3/ms-

OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacity of 44.42 GiB of which 314.06 MiB is free. Process 4027711 has 38.43 GiB memory in use. Including non-PyTorch memory, this process has 5.67 GiB memory in use. Of the allocated memory 4.61 GiB is allocated by PyTorch, and 571.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
pipe.load_textual_inversion("words")

In [ ]:
from diffusers import StableDiffusionPipeline


pipe2 = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")